# *A Student's Guide to the Mathematics of Astronomy*

This is an F# notebook implementing the concepts found in the book [*A Student's Guide to the Mathematics of Astronomy* by Daniel Fleisch and Julia Kregenow](https://www.danfleisch.com/sgmoa/).

In [12]:
open FSharp.Data.UnitSystems.SI.UnitSymbols

We will define all units of measure here. See [Units of Measure](https://learn.microsoft.com/en-us/dotnet/fsharp/language-reference/units-of-measure).

In [13]:
[<Measure>] type inch
[<Measure>] type feet
[<Measure>] type hour
[<Measure>] type minute
[<Measure>] type lightyear
[<Measure>] type parsec
[<Measure>] type cm
[<Measure>] type km
[<Measure>] type mile
[<Measure>] type arcmin
[<Measure>] type arcsec
[<Measure>] type degree
[<Measure>] type second
[<Measure>] type year
[<Measure>] type day

A `ConversionRelation` type is defined simply as a name for a tuple of floats of the two units. Generic units of measure types are used in this type definition. See [Names of Tuple Types](https://learn.microsoft.com/en-us/dotnet/fsharp/language-reference/tuples#names-of-tuple-types) and [Generics](https://learn.microsoft.com/en-us/dotnet/fsharp/language-reference/generics/).

In [14]:
/// Represents a conversion relation that relates a value in one unit
/// of measure to an equivalent value in another unit of measure
type ConversionRelation<[<Measure>] 'Unit1, [<Measure>] 'Unit2> =
    float<'Unit1> * float<'Unit2>

It is now easy to build conversion factors from a conversion relation and functions to convert from unit to the other.

In [15]:
/// Returns a conversion factor than can be used to multiply a value in `Unit1` to
/// convert it to a value in `Unit2`
let getUnit1ToUnit2ConversionFactor<[<Measure>] 'Unit1, [<Measure>]'Unit2> (conversionRelation: ConversionRelation<'Unit1, 'Unit2>) =
    let (a, b) = conversionRelation
    b / a

/// Returns a conversion factor than can be used to multiply a value in `Unit2` to
/// convert it to a value in `Unit1`
let getUnit2ToUnit1ConversionFactor<[<Measure>] 'Unit1, [<Measure>]'Unit2> (conversionRelation: ConversionRelation<'Unit1, 'Unit2>) =
    let (a, b) = conversionRelation
    a / b

/// Given a conversion relation, convert any value in `Unit1` to a value in `Unit2`
let convertUnit1ToUnit2<[<Measure>] 'Unit1, [<Measure>]'Unit2> (conversionRelation: ConversionRelation<'Unit1, 'Unit2>) (value: float<'Unit1>) =
    value * (getUnit1ToUnit2ConversionFactor conversionRelation)

/// Given a conversion relation, convert any value in `Unit2` to a value in `Unit1`
let convertUnit2ToUnit1<[<Measure>] 'Unit1, [<Measure>]'Unit2> (conversionRelation: ConversionRelation<'Unit1, 'Unit2>) (value: float<'Unit2>) =
    value * (getUnit2ToUnit1ConversionFactor conversionRelation)

In [16]:
let inchAndFeetConversionRelation = (12.0<inch>, 1.0<feet>)
let hourAndMinuteConversionRelation = (1.0<hour>, 60.0<minute>)
let lightyearAndParsecConversionRelation = (3.26<lightyear>, 1.0<parsec>)

let convertInchesToFeet inches =
    convertUnit1ToUnit2<inch, feet> inchAndFeetConversionRelation inches

let convertFeetToInches feet =
    convertUnit2ToUnit1<inch, feet> inchAndFeetConversionRelation feet

#### Exercise 1.1

Write the following equivalence relations as fractional conversion factors:
* 1 in <-> 2.54 cm
* 1.6 km <-> 1 mile
* 60 arcmin <-> 3,600 arcsec

In [17]:
let inchAndCentimetersConversionRelation = (1.0<inch>, 2.54<cm>)
let kmAndMileConversionRelation = (1.6<km>, 1.0<mile>)
let arcminAndArcsecConversionRelation = (60.0<arcmin>, 3_600.0<arcsec>)

See [Plain text formatting](https://learn.microsoft.com/en-us/dotnet/fsharp/language-reference/plaintext-formatting).

*Example: Convert 1,000 minutes to hours.*

In [18]:
convertUnit2ToUnit1 lightyearAndParsecConversionRelation 1.29<parsec>
|> printfn "%A light years"

4.2054 light years


*Example: Convert 1.29 parsecs, the distance of the closest star beyond our
Sun, to light years.*

In [19]:
convertUnit2ToUnit1 hourAndMinuteConversionRelation 1_000.0<minute>
|> printfn "%A hours"

16.66666667 hours


#### Exercise 1.2

<ol type="a">
    <li>Express 12 inches in centimeters.</li>
    <li>Express 100 cm in inches.</li>
    <li>Express 380,000 km in miles (this is roughly the distance from the
Earth to the Moon).</li>
    <li>Express 93,000,000 miles in kilometers (this is roughly the distance
from the Earth to the Sun).</li>
    <li>Express 0.5 degrees in arcseconds (this is roughly the angular size of
the full Moon viewed from Earth).</li>
</ol>

In [20]:
// (a)
convertUnit1ToUnit2 inchAndCentimetersConversionRelation 12.0<inch>
|> printfn "(a) %A cm"

// (b)
convertUnit2ToUnit1 inchAndCentimetersConversionRelation 100.0<cm>
|> printfn "(b) %A inches"

// (c)
convertUnit1ToUnit2 kmAndMileConversionRelation 380_000.0<km>
|> printfn "(c) %A miles"

// (d)
convertUnit2ToUnit1 kmAndMileConversionRelation 93_000_000.0<mile>
|> printfn "(d) %A km"

// (e)
let degreeAndArcsecondConversionRelation = (1.0<degree>, 3_600.0<arcsec>)
convertUnit1ToUnit2 degreeAndArcsecondConversionRelation 0.5<degree>
|> printfn "(e) %A arcseconds"

(a) 30.48 cm
(b) 39.37007874 inches
(c) 237500.0 miles
(d) 148800000.0 km
(e) 1800.0 arcseconds


#### Exercise 1.3

How do you know that your answers to each of the unit
conversion problems in the previous exercise make sense? Give a brief
explanation for each.

**Answer**: Because this is F#, and it makes unit conversions nearly trivial. :) A more thorough answer is that we have defined units of measure as types in F#, then conversion relations as ordered tuples of units of measure, and then defined F# functions that allow us to easily build conversion factors for the conversion relations and functions that convert from unit to the other. F# keeps track of the units, and so it would warn us if we tried to convert using the wrong units. F# effectively ensures the checks described in Sections **1.1.2 Setting up a conversion problem** and **1.1.3 Checking your answer** via its type system.

*Example: Convert from kilometers per hour to meters per second.*

In [21]:
(convertUnit1ToUnit2 (1.0<km>, 1000.0<m>) 1.0<km>)
/ (convertUnit1ToUnit2 (1.0<hour>, 3_600.0<second>) 1.0<hour>)

0.2777777777777778

*Example: How many seconds old were you on your first birthday?*

In [22]:
let convertYearsToSeconds years =
    convertUnit1ToUnit2 (1.0<year>, 365.0<day>) years
    |> convertUnit1ToUnit2 (1.0<day>, 24.0<hour>)
    |> convertUnit1ToUnit2 (1.0<hour>, 60.0<minute>)
    |> convertUnit1ToUnit2 (1.0<minute>, 60.0<second>)

convertYearsToSeconds 1.0<year>
|> printfn "1 year is %A seconds"

convertYearsToSeconds 30.0<year>
|> printfn "30 years is %A seconds"

1 year is 31536000.0 seconds
30 years is 946080000.0 seconds


#### Exercise 1.4. Perform the following unit conversions.

<ol type="a">
    <li>Convert 60 mph (miles per hour) to meters per second.</li>
    <li>Convert 1 day to seconds.</li>
    <li>Convert dollars per kilogram to cents per gram (100 cents ↔ 1 dollar).</li>
    <li>Convert 1 mile to steps, assuming 1 step ↔ 30 inches (there are 1,760
    yards in 1 mile, 3 ft in 1 yard, and 12 inches in 1 ft).</li>
</ol>